# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

### 데이터 셋 읽어오기

In [36]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

결측치 개수 출력

In [23]:
df_train = df_train[['ver_cus','id_strategic_ver','is_converted']]

#df_train
df_train


,ver_cus,id_strategic_ver,is_converted
0,1,NaN,True
1,1,NaN,True
2,1,NaN,True
3,1,NaN,True
4,0,NaN,True
...,...,...,...
59294,0,NaN,False
59295,0,NaN,False
59296,0,NaN,False
59297,0,NaN,False


In [24]:
df_test=df_test[['id_strategic_ver','ver_cus']]
df_test

,id_strategic_ver,ver_cus
0,NaN,1
1,NaN,0
2,NaN,0
3,NaN,0
4,1.0,1
...,...,...
5266,NaN,0
5267,NaN,0
5268,NaN,0
5269,NaN,0


In [13]:
#각 열별 결측치 개수 출력
missing_values = df_train.isnull().sum()
# 결과 출력
print("각 열의 결측치 개수:")
print(missing_values)


# # 특정 컬럼 리스트 지정
# column_list = ['ver_win_ratio_per_bu', 'business_area']

# # 지정된 컬럼들의 결측치 개수 출력
# missing_values_specific_columns = df_train[column_list].isnull().sum()

# # 결과 출력
# print("특정 컬럼들의 결측치 개수:")
# print(missing_values_specific_columns)

각 열의 결측치 개수:
bant_submit                    0
customer_country             982
business_unit                  0
com_reg_ver_win_rate       44731
customer_idx                   0
customer_type              43961
enterprise                     0
historical_existing_cnt    45543
id_strategic_ver           55855
it_strategic_ver           58178
idit_strategic_ver         54734
customer_job               18733
lead_desc_length               0
inquiry_type                 941
product_category           19374
product_subcategory        50064
product_modelname          50070
customer_country.1           982
customer_position              0
response_corporate             0
expected_timeline          30863
ver_cus                        0
ver_pro                        0
ver_win_rate_x             40882
ver_win_ratio_per_bu       43995
business_area              40882
business_subarea           53773
lead_owner                     0
is_converted                   0
dtype: int64


In [4]:
target_rows = df_train[(df_train['is_converted'] == True) & (df_train['business_area'] == 'ID')]
rest_columns = df_train.columns.difference(['is_converted', 'business_area'])

for column in rest_columns:
    unique_values_count = target_rows[column].nunique()
    print(f"{column}의 유니크한 값들의 개수: {unique_values_count}")

bant_submit 칼럼의 유니크한 값들의 개수: 0
customer_country 칼럼의 유니크한 값들의 개수: 0
business_unit 칼럼의 유니크한 값들의 개수: 0
com_reg_ver_win_rate 칼럼의 유니크한 값들의 개수: 0
customer_idx 칼럼의 유니크한 값들의 개수: 0
customer_type 칼럼의 유니크한 값들의 개수: 0
enterprise 칼럼의 유니크한 값들의 개수: 0
historical_existing_cnt 칼럼의 유니크한 값들의 개수: 0
id_strategic_ver 칼럼의 유니크한 값들의 개수: 0
it_strategic_ver 칼럼의 유니크한 값들의 개수: 0
idit_strategic_ver 칼럼의 유니크한 값들의 개수: 0
customer_job 칼럼의 유니크한 값들의 개수: 0
lead_desc_length 칼럼의 유니크한 값들의 개수: 0
inquiry_type 칼럼의 유니크한 값들의 개수: 0
product_category 칼럼의 유니크한 값들의 개수: 0
product_subcategory 칼럼의 유니크한 값들의 개수: 0
product_modelname 칼럼의 유니크한 값들의 개수: 0
customer_country.1 칼럼의 유니크한 값들의 개수: 0
customer_position 칼럼의 유니크한 값들의 개수: 0
response_corporate 칼럼의 유니크한 값들의 개수: 0
expected_timeline 칼럼의 유니크한 값들의 개수: 0
ver_cus 칼럼의 유니크한 값들의 개수: 0
ver_pro 칼럼의 유니크한 값들의 개수: 0
ver_win_rate_x 칼럼의 유니크한 값들의 개수: 0
ver_win_ratio_per_bu 칼럼의 유니크한 값들의 개수: 0
business_subarea 칼럼의 유니크한 값들의 개수: 0
lead_owner 칼럼의 유니크한 값들의 개수: 0


In [29]:
# condition=df_train[(df_train['business_unit']=='ID')&(df_train['id_strategic_ver']==1)].shape[0]
condition=df_train[(df_train['business_unit']=='ID')&(df_train['id_strategic_ver']==1)&(df_train['is_converted']==True)].shape[0]
print(condition)
# # business_unit이 'id'이고 id_strategic_ver이 1이며 is_converted가 True인 행의 개수 출력
# condition = (df_train['business_unit'] == 'ID') & (df_train['id_strategic_ver'] == 1) & (df_train['is_converted'] == True)
# count_matching_rows = df[condition].shape[0]

# print(f"조건에 해당하는 행의 개수: {count_matching_rows}")


335


In [35]:
# unique_values_count = df_train['business_area'].nunique()
# print(f"business_area 칼럼의 고유한 값들의 개수: {unique_values_count}")

unique_values = df_train['business_area'].unique()

print("business_area 칼럼의 고유한 값들:")
for value in unique_values:
    print(value)

business_area 칼럼의 고유한 값들:
corporate / office
nan
education
hotel & accommodation
hospital & health care
special purpose
residential (home)
government department
retail
factory
power plant / renewable energy
transportation
public facility


중복 컬럼 드랍하기

In [3]:
# #드롭하고자 하는 컬럼들의 리스트
# #columns_to_drop = ['id_strategic_ver', 'it_strategic_ver','idit_strategic_ver','product_subcategory','product_modelname','business_subarea']
# #columns_to_drop = ['id_strategic_ver', 'it_strategic_ver','idit_strategic_ver']
# columns_to_drop = ['product_modelname','product_subcategory','business_subarea']

# # DataFrame에서 컬럼들을 드롭
# df_train = df_train.drop(columns=columns_to_drop, axis=1)
# df_test  = df_test.drop(columns=columns_to_drop, axis=1)

df_train=df_train.drop(columns="customer_country.1")
df_test=df_test.drop(columns="customer_country.1")



과거 영업 전환 이력이 있으면 1로 설정

In [9]:
# # historical_existing_cnt 컬럼의 NaN 값을 0으로 채우기
# df_train['historical_existing_cnt'].fillna(0, inplace=True)
# df_test['historical_existing_cnt'].fillna(0, inplace=True)
# df_train


# NaN 값을 0으로 채우기
df_train['historical_existing_cnt'].fillna(0, inplace=True)
df_test['historical_existing_cnt'].fillna(0, inplace=True)

# 0이 아니면 1로 레이블링
df_train['historical_existing_cnt'] = df_train['historical_existing_cnt'].apply(lambda x: 1 if x != 0 else 0)
df_test['historical_existing_cnt'] = df_test['historical_existing_cnt'].apply(lambda x: 1 if x != 0 else 0)

/tmp/ipykernel_154/823475726.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['historical_existing_cnt'].fillna(0, inplace=True)
/tmp/ipykernel_154/823475726.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,1,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,0,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,0,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,0,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,1,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,1,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,False
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,False
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,0,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,False
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,0,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,False


## 2. 데이터 전처리

### 레이블 인코딩

In [4]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [5]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    # "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [6]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [20]:
target_rows = df_train[(df_train['is_converted'] == True)&(df_train['ver_cus']==0) ].shape[0]
print(target_rows)

4442


In [58]:

# target_condition = (df_train['is_converted'] == True)) & (df_train['business_unit'] == 'ID')
filtered_data = df_train[target_condition]
df_train[df_train['business_unit'] == 0]
# df_train[filtered_data['historical_existing_cnt']]
# 나머지 컬럼들의 값들의 유니크한 개수 출력
# for column in df_train.columns:
#     if column not in ['is_converted', 'business_unit']:
#         unique_values_count = df_train[filtered_data[column]].nunique()
#         print(f"{column} 칼럼의 유니크한 값들의 개수: {unique_values_count}")
df_train


,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,9070,0,0.066667,32160,10,0,NaN,NaN,NaN,...,33,246,1,0,0.003079,0.026846,0,28,0,True
1,1.00,8406,0,0.066667,23122,10,0,12.0,NaN,NaN,...,33,246,1,0,0.003079,0.026846,0,0,1,True
2,1.00,6535,0,0.088889,1755,10,0,144.0,NaN,NaN,...,21,246,1,0,0.003079,0.026846,0,17,2,True
3,1.00,3388,0,0.088889,4919,10,0,NaN,NaN,NaN,...,21,246,1,0,0.003079,0.026846,0,44,3,True
4,1.00,5799,0,0.088889,17126,29,0,NaN,NaN,NaN,...,21,246,0,0,0.003079,0.026846,0,86,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,10287,0,NaN,33747,9,1,NaN,NaN,NaN,...,34,8,0,0,0.000026,0.028777,8,62,694,False
59295,0.75,3490,0,0.040000,35420,29,0,NaN,NaN,NaN,...,7,15,0,0,0.000026,0.028777,8,86,39,False
59296,0.75,8799,0,0.040000,19249,29,0,NaN,NaN,NaN,...,35,246,0,0,0.000026,0.028777,8,86,125,False
59297,1.00,12795,0,0.040000,40327,33,0,NaN,NaN,NaN,...,35,267,0,0,0.000026,0.028777,8,86,134,False


In [ ]:
# historical_existing_cnt가 0이 아니고 is_converted가 True인 행들의 개수 출력
non_zero_and_converted_count = ((df_train['historical_existing_cnt'] != 0) & (df_train['is_converted'] == True)).sum()
non_zero_and_not_converted_count    = ((df_train['historical_existing_cnt'] != 0) & (df_train['is_converted'] == False)).sum()
print(f'`historical_existing_cnt`가 0이 아니고 `is_converted`가 True인 행들의 개수: {non_zero_and_converted_count}')
print(f'`historical_existing_cnt`가 0이 아니고 `is_converted`가 False인 행들의 개수: {non_zero_and_not_converted_count}')

### 2-2. 학습, 검증 데이터 분리

In [7]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [25]:
df_train
x_train = df_train[['id_strategic_ver','ver_cus']]
x_train

,id_strategic_ver,ver_cus
0,NaN,1
1,NaN,1
2,NaN,1
3,NaN,1
4,NaN,0
...,...,...
59294,NaN,0
59295,NaN,0
59296,NaN,0
59297,NaN,0


In [26]:
y_train = df_train['is_converted']
y_train

0         True
1         True
2         True
3         True
4         True
         ...  
59294    False
59295    False
59296    False
59297    False
59298    False
Name: is_converted, Length: 59299, dtype: bool

## 3. 모델 학습

### 모델 정의 

In [27]:
model = DecisionTreeClassifier()

# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()

# from xgboost import XGBClassifier       
# model = XGBClassifier()

# from lightgbm import LGBMClassifier
# model = LGBMClassifier()

### 모델 학습

In [20]:
pred
for i in pred:
    if i == True:
        print(i)

In [31]:
x_train = x_train.fillna(0)
model.fit(x_train, y_train)

DecisionTreeClassifier()

### 모델 성능 보기

In [29]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [32]:
pred = model.predict(x_train.fillna(0))
get_clf_eval(y_train, pred)

오차행렬:
 [[    0  4850]
 [    0 54449]]

정확도: 0.9182
정밀도: 0.0000
재현율: 0.0000
F1: 0.0000


/home/elicer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 4. 제출하기

### 테스트 데이터 예측

In [19]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [30]:
test_pred = model.predict(x_test.fillna(0))
sum(test_pred) # True로 예측된 개수

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- customer_country.1


In [39]:
df_test.loc[df_test['ver_cus'] == 0, 'is_converted'] = 1

df_test

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,1
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,1
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,1
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,1
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,1
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,1
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,1
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,1


In [42]:
df_test.loc[df_test['ver_cus'] != 0, 'is_converted'] = 0
df_test

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,0
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,1
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,1
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,1
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,1
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,1
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,1
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,1


### 제출 파일 작성

In [43]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = df_test['is_converted']

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**

In [44]:
sum(df_test['is_converted']) # True로 예측된 개수

4497